In [147]:
import re
import requests  
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import HTML
import sys 
sys.setrecursionlimit(10000000) 
pd.set_option('display.max_colwidth', 10000000)
nltk.download('stopwords')
%matplotlib inline
import numpy as np
import scipy as sp

import sklearn
from sklearn import datasets, feature_selection, cluster, feature_extraction
from sklearn import neighbors, decomposition, metrics, preprocessing
from sklearn import cross_validation, grid_search #model_selection

#from sklearn import cluster
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import MDS


# do conda install nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from IPython import display

# Do conda install pillow
from PIL import Image
import os
import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

# HTML('<iframe src=http://shakespeare.mit.edu/ width=700 height=500></iframe>')
# special matplotlib command for global plot configuration
from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.colors import ListedColormap

dark2_colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d','#666666']
set2_colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f','#e5c494','#b3b3b3']

bold_colors = ['#FF0000', '#00FF00', '#FFAAAA', '#0000FF']
light_colors = ['#FFAAAA', '#AAFFAA', '#AAAAFF', '#00FF00']
cmap_light = ListedColormap(light_colors)
cmap_bold = ListedColormap(bold_colors)
dark2_cmap = ListedColormap(dark2_colors)
set2_cmap = ListedColormap(dark2_colors)
light_grey = np.array([float(248)/float(255)]*3)
shade_black = '#262626'

def set_mpl_params():
    rcParams['figure.figsize'] = (10, 6)
    rcParams['figure.dpi'] = 150
    rcParams['axes.prop_cycle'].by_key()['color'][1]
    rcParams['lines.linewidth'] = 2
    rcParams['axes.facecolor'] = 'white'
    rcParams['font.size'] = 16
    rcParams['patch.edgecolor'] = 'white'
    rcParams['patch.facecolor'] = dark2_colors[0]
    rcParams['font.family'] = 'StixGeneral'

set_mpl_params()

# Remove border function taken from CS109 2013 
######################################
def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecessary plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()
        
def cible_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Make a target axis at 0,0 with ticks along the axis lines
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()

    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.spines['bottom'].set_position(('data',0))
    ax.yaxis.set_ticks_position('left')
    ax.spines['left'].set_position(('data',0))

    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Jeannie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Get Data

In [2]:
BASE_URL = 'http://shakespeare.mit.edu/'
r = requests.get(BASE_URL)
assert(r.status_code == 200)
soup = BeautifulSoup(r.content, 'html.parser')
table = soup.findAll('table')[1]

def genres():
    trs_genre = table.findAll('h2')                 # GET GENRE IN EACH TABLE
    return [tr.text.strip() for tr in trs_genre if tr.text not in '\nPoetry']
genres()

['Comedy', 'History', 'Tragedy']

### Play by genre

In [15]:
def plays(genre):
    getindex = (['Comedy', 'History', 'Tragedy'].index(genre))
    table = soup.findAll('table')[1]
    trs_genre = table.find_all('td', {'valign': 'BASELINE'})
    cnt = 0
    dict_value = {}
    dict_link = {}
    for tr in trs_genre:
        link_trs = tr.find_all('a', href=True)      # GET LINK WITH IN EACH GENRE        
        for tr in link_trs:
            link = BASE_URL + tr['href']  
            value = ''.join(tr.contents).strip()       
            dict_link.setdefault(cnt,[]).append(link)
            dict_value.setdefault(cnt,[]).append(value)
        cnt = cnt +1      
    return dict_value.get(getindex) , dict_link.get(getindex)

plays('Tragedy')
# plays('Comedy')[0]
# plays('Comedy')[1]
# plays('History')

(['Antony and Cleopatra',
  'Coriolanus',
  'Hamlet',
  'Julius Caesar',
  'King Lear',
  'Macbeth',
  'Othello',
  'Romeo and Juliet',
  'Timon of Athens',
  'Titus Andronicus'],
 ['http://shakespeare.mit.edu/cleopatra/index.html',
  'http://shakespeare.mit.edu/coriolanus/index.html',
  'http://shakespeare.mit.edu/hamlet/index.html',
  'http://shakespeare.mit.edu/julius_caesar/index.html',
  'http://shakespeare.mit.edu/lear/index.html',
  'http://shakespeare.mit.edu/macbeth/index.html',
  'http://shakespeare.mit.edu/othello/index.html',
  'http://shakespeare.mit.edu/romeo_juliet/index.html',
  'http://shakespeare.mit.edu/timon/index.html',
  'http://shakespeare.mit.edu/titus/index.html'])

### Get all scene links by play

In [9]:
def act_scene(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html.parser')
    paragraph = soup.findAll('p')[2]

    links = [BASE_URL + tag['href'].split(".")[0] + "/" + tag['href'] for tag in paragraph.select('a[href]')]
    values = [tag.getText() for tag in paragraph.select('a[href]')]
    return  values , links

act_scene('http://shakespeare.mit.edu/allswell/index.html')[1]

['http://shakespeare.mit.edu/allswell/allswell.1.1.html',
 'http://shakespeare.mit.edu/allswell/allswell.1.2.html',
 'http://shakespeare.mit.edu/allswell/allswell.1.3.html',
 'http://shakespeare.mit.edu/allswell/allswell.2.1.html',
 'http://shakespeare.mit.edu/allswell/allswell.2.2.html',
 'http://shakespeare.mit.edu/allswell/allswell.2.3.html',
 'http://shakespeare.mit.edu/allswell/allswell.2.4.html',
 'http://shakespeare.mit.edu/allswell/allswell.2.5.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.1.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.2.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.3.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.4.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.5.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.6.html',
 'http://shakespeare.mit.edu/allswell/allswell.3.7.html',
 'http://shakespeare.mit.edu/allswell/allswell.4.1.html',
 'http://shakespeare.mit.edu/allswell/allswell.4.2.html',
 'http://shake

### Get line by scene

In [86]:

def sceneline(URL): 
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    records = [] 
    speeches = soup.findAll(attrs={'name': re.compile('speech\d+')})
    
    for i in list(range(len(speeches))): 
        character = soup.findAll(attrs={'name': re.compile('speech\d+')})[i].string
        try:
            blockquotes = soup.find_all(lambda tag: tag.name == "blockquote" and tag.a)[i]
        except IndexError:
#             print(i,speeches,URL,"bq Index ERROR ")
            pass
        quotes = blockquotes.findAll('a')

        for bq in quotes:
            record = []
            record.append(URL)
            record.append(i+1)
            record.append(character)
            record.append(blockquotes.text.strip())
            try:
                record.append(bq['name'])
            except KeyError:
#                 print(i,bq,URL,"bq KEY ERROR ")
                pass

            record.append(bq.text)
            records.append(record)           
    
#     return records
    df = pd.DataFrame(data=records)
    return(df)

# type(scene('http://shakespeare.mit.edu/allswell/allswell.1.1.html'))
sceneline('http://shakespeare.mit.edu/allswell/allswell.1.2.html')


,0,1,2,3,4,5
0,http://shakespeare.mit.edu/allswell/allswell.1.2.html,1,KING,The Florentines and Senoys are by the ears;\nHave fought with equal fortune and continue\nA braving war.,1,The Florentines and Senoys are by the ears;
1,http://shakespeare.mit.edu/allswell/allswell.1.2.html,1,KING,The Florentines and Senoys are by the ears;\nHave fought with equal fortune and continue\nA braving war.,2,Have fought with equal fortune and continue
2,http://shakespeare.mit.edu/allswell/allswell.1.2.html,1,KING,The Florentines and Senoys are by the ears;\nHave fought with equal fortune and continue\nA braving war.,3,A braving war.
3,http://shakespeare.mit.edu/allswell/allswell.1.2.html,2,First Lord,"So 'tis reported, sir.",4,"So 'tis reported, sir."
4,http://shakespeare.mit.edu/allswell/allswell.1.2.html,3,KING,"Nay, 'tis most credible; we here received it\nA certainty, vouch'd from our cousin Austria,\nWith caution that the Florentine will move us\nFor speedy aid; wherein our dearest friend\nPrejudicates the business and would seem\nTo have us make denial.",5,"Nay, 'tis most credible; we here received it"
5,http://shakespeare.mit.edu/allswell/allswell.1.2.html,3,KING,"Nay, 'tis most credible; we here received it\nA certainty, vouch'd from our cousin Austria,\nWith caution that the Florentine will move us\nFor speedy aid; wherein our dearest friend\nPrejudicates the business and would seem\nTo have us make denial.",6,"A certainty, vouch'd from our cousin Austria,"
6,http://shakespeare.mit.edu/allswell/allswell.1.2.html,3,KING,"Nay, 'tis most credible; we here received it\nA certainty, vouch'd from our cousin Austria,\nWith caution that the Florentine will move us\nFor speedy aid; wherein our dearest friend\nPrejudicates the business and would seem\nTo have us make denial.",7,With caution that the Florentine will move us
7,http://shakespeare.mit.edu/allswell/allswell.1.2.html,3,KING,"Nay, 'tis most credible; we here received it\nA certainty, vouch'd from our cousin Austria,\nWith caution that the Florentine will move us\nFor speedy aid; wherein our dearest friend\nPrejudicates the business and would seem\nTo have us make denial.",8,For speedy aid; wherein our dearest friend
8,http://shakespeare.mit.edu/allswell/allswell.1.2.html,3,KING,"Nay, 'tis most credible; we here received it\nA certainty, vouch'd from our cousin Austria,\nWith caution that the Florentine will move us\nFor speedy aid; wherein our dearest friend\nPrejudicates the business and would seem\nTo have us make denial.",9,Prejudicates the business and would seem
9,http://shakespeare.mit.edu/allswell/allswell.1.2.html,3,KING,"Nay, 'tis most credible; we here received it\nA certainty, vouch'd from our cousin Austria,\nWith caution that the Florentine will move us\nFor speedy aid; wherein our dearest friend\nPrejudicates the business and would seem\nTo have us make denial.",10,To have us make denial.


### Create the first table prepared to merge

In [71]:
def database_first():
    records = []
    for genre in genres():
        for play,play_link in zip(plays(genre)[0],plays(genre)[1]):        
            for scene,scene_link in zip(act_scene(play_link)[0],act_scene(play_link)[1]):
                record = []
                record.append(genre)
                record.append(play)  
                record.append(play_link)
                record.append(scene)  
                record.append(scene_link)

                records.append(record)
                
    df = pd.DataFrame(data=records)
        
    return(df)
df1 = database_first()
df1.head()

,0,1,2,3,4
0,Comedy,All's Well That Ends Well,http://shakespeare.mit.edu/allswell/index.html,Rousillon. The COUNT's palace.,http://shakespeare.mit.edu/allswell/allswell.1.1.html
1,Comedy,All's Well That Ends Well,http://shakespeare.mit.edu/allswell/index.html,Paris. The KING's palace.,http://shakespeare.mit.edu/allswell/allswell.1.2.html
2,Comedy,All's Well That Ends Well,http://shakespeare.mit.edu/allswell/index.html,Rousillon. The COUNT's palace.,http://shakespeare.mit.edu/allswell/allswell.1.3.html
3,Comedy,All's Well That Ends Well,http://shakespeare.mit.edu/allswell/index.html,Paris. The KING's palace.,http://shakespeare.mit.edu/allswell/allswell.2.1.html
4,Comedy,All's Well That Ends Well,http://shakespeare.mit.edu/allswell/index.html,Rousillon. The COUNT's palace.,http://shakespeare.mit.edu/allswell/allswell.2.2.html


In [42]:
df1.shape

(753, 5)

In [89]:
df1.columns = ['genre', 'play','playindex','actscene','actscenelink']
df1.columns
df1['actscenelink'] = df1['actscenelink'].map(str.strip)
# TEST
df2 = df1[:5]
df2['actscenelink']

0    http://shakespeare.mit.edu/allswell/allswell.1.1.html
1    http://shakespeare.mit.edu/allswell/allswell.1.2.html
2    http://shakespeare.mit.edu/allswell/allswell.1.3.html
3    http://shakespeare.mit.edu/allswell/allswell.2.1.html
4    http://shakespeare.mit.edu/allswell/allswell.2.2.html
Name: actscenelink, dtype: object

### Create second dataset for all line in each scence

In [111]:
df3 = pd.DataFrame()
for i in df1['actscenelink']:
    print(i)
    URL = ''.join(i).strip()     
    data = sceneline(URL)
    print("data = ",data.shape)
    df3 = df3.append(data)
    print("df3 = ", df3.shape)
    
df3.head()

http://shakespeare.mit.edu/allswell/allswell.1.1.html
data =  (222, 6)
df3 =  (222, 6)
http://shakespeare.mit.edu/allswell/allswell.1.2.html
data =  (86, 6)
df3 =  (308, 6)
http://shakespeare.mit.edu/allswell/allswell.1.3.html
data =  (266, 6)
df3 =  (574, 6)
http://shakespeare.mit.edu/allswell/allswell.2.1.html
data =  (222, 6)
df3 =  (796, 6)
http://shakespeare.mit.edu/allswell/allswell.2.2.html
data =  (64, 6)
df3 =  (860, 6)
http://shakespeare.mit.edu/allswell/allswell.2.3.html
data =  (302, 6)
df3 =  (1162, 6)
http://shakespeare.mit.edu/allswell/allswell.2.4.html
data =  (56, 6)
df3 =  (1218, 6)
http://shakespeare.mit.edu/allswell/allswell.2.5.html
data =  (100, 6)
df3 =  (1318, 6)
http://shakespeare.mit.edu/allswell/allswell.3.1.html
data =  (27, 6)
df3 =  (1345, 6)
http://shakespeare.mit.edu/allswell/allswell.3.2.html
data =  (135, 6)
df3 =  (1480, 6)
http://shakespeare.mit.edu/allswell/allswell.3.3.html
data =  (14, 6)
df3 =  (1494, 6)
http://shakespeare.mit.edu/allswell/allswe

,0,1,2,3,4,5
0,http://shakespeare.mit.edu/allswell/allswell.1.1.html,1,COUNTESS,"In delivering my son from me, I bury a second husband.",1,"In delivering my son from me, I bury a second husband."
1,http://shakespeare.mit.edu/allswell/allswell.1.1.html,2,BERTRAM,"And I in going, madam, weep o'er my father's death\nanew: but I must attend his majesty's command, to\nwhom I am now in ward, evermore in subjection.",2,"And I in going, madam, weep o'er my father's death"
2,http://shakespeare.mit.edu/allswell/allswell.1.1.html,2,BERTRAM,"And I in going, madam, weep o'er my father's death\nanew: but I must attend his majesty's command, to\nwhom I am now in ward, evermore in subjection.",3,"anew: but I must attend his majesty's command, to"
3,http://shakespeare.mit.edu/allswell/allswell.1.1.html,2,BERTRAM,"And I in going, madam, weep o'er my father's death\nanew: but I must attend his majesty's command, to\nwhom I am now in ward, evermore in subjection.",4,"whom I am now in ward, evermore in subjection."
4,http://shakespeare.mit.edu/allswell/allswell.1.1.html,3,LAFEU,"You shall find of the king a husband, madam; you,\nsir, a father: he that so generally is at all times\ngood must of necessity hold his virtue to you; whose\nworthiness would stir it up where it wanted rather\nthan lack it where there is such abundance.",5,"You shall find of the king a husband, madam; you,"


### Merge data to get final dataset 

In [119]:
df3.columns = ['actscenelink','speechnumber','character','blockquotes','linenumber','line']
df_all = pd.merge(df1, df3, on='actscenelink', how='left')
df_all.head()
df_all.tail()

,genre,play,playindex,actscene,actscenelink,speechnumber,character,blockquotes,linenumber,line
108731,Tragedy,Titus Andronicus,http://shakespeare.mit.edu/titus/index.html,Court of TITUS's house. A banquet set out.,http://shakespeare.mit.edu/titus/titus.5.3.html,43.0,LUCIUS,"Some loving friends convey the emperor hence,\nAnd give him burial in his father's grave:\nMy father and Lavinia shall forthwith\nBe closed in our household's monument.\nAs for that heinous tiger, Tamora,\nNo funeral rite, nor man in mourning weeds,\nNo mournful bell shall ring her burial;\nBut throw her forth to beasts and birds of prey:\nHer life was beast-like, and devoid of pity;\nAnd, being so, shall have like want of pity.\nSee justice done on Aaron, that damn'd Moor,\nBy whom our heavy haps had their beginning:\nThen, afterwards, to order well the state,\nThat like events may ne'er it ruinate.\nExeunt",201,"And, being so, shall have like want of pity."
108732,Tragedy,Titus Andronicus,http://shakespeare.mit.edu/titus/index.html,Court of TITUS's house. A banquet set out.,http://shakespeare.mit.edu/titus/titus.5.3.html,43.0,LUCIUS,"Some loving friends convey the emperor hence,\nAnd give him burial in his father's grave:\nMy father and Lavinia shall forthwith\nBe closed in our household's monument.\nAs for that heinous tiger, Tamora,\nNo funeral rite, nor man in mourning weeds,\nNo mournful bell shall ring her burial;\nBut throw her forth to beasts and birds of prey:\nHer life was beast-like, and devoid of pity;\nAnd, being so, shall have like want of pity.\nSee justice done on Aaron, that damn'd Moor,\nBy whom our heavy haps had their beginning:\nThen, afterwards, to order well the state,\nThat like events may ne'er it ruinate.\nExeunt",202,"See justice done on Aaron, that damn'd Moor,"
108733,Tragedy,Titus Andronicus,http://shakespeare.mit.edu/titus/index.html,Court of TITUS's house. A banquet set out.,http://shakespeare.mit.edu/titus/titus.5.3.html,43.0,LUCIUS,"Some loving friends convey the emperor hence,\nAnd give him burial in his father's grave:\nMy father and Lavinia shall forthwith\nBe closed in our household's monument.\nAs for that heinous tiger, Tamora,\nNo funeral rite, nor man in mourning weeds,\nNo mournful bell shall ring her burial;\nBut throw her forth to beasts and birds of prey:\nHer life was beast-like, and devoid of pity;\nAnd, being so, shall have like want of pity.\nSee justice done on Aaron, that damn'd Moor,\nBy whom our heavy haps had their beginning:\nThen, afterwards, to order well the state,\nThat like events may ne'er it ruinate.\nExeunt",203,By whom our heavy haps had their beginning:
108734,Tragedy,Titus Andronicus,http://shakespeare.mit.edu/titus/index.html,Court of TITUS's house. A banquet set out.,http://shakespeare.mit.edu/titus/titus.5.3.html,43.0,LUCIUS,"Some loving friends convey the emperor hence,\nAnd give him burial in his father's grave:\nMy father and Lavinia shall forthwith\nBe closed in our household's monument.\nAs for that heinous tiger, Tamora,\nNo funeral rite, nor man in mourning weeds,\nNo mournful bell shall ring her burial;\nBut throw her forth to beasts and birds of prey:\nHer life was beast-like, and devoid of pity;\nAnd, being so, shall have like want of pity.\nSee justice done on Aaron, that damn'd Moor,\nBy whom our heavy haps had their beginning:\nThen, afterwards, to order well the state,\nThat like events may ne'er it ruinate.\nExeunt",204,"Then, afterwards, to order well the state,"
108735,Tragedy,Titus Andronicus,http://shakespeare.mit.edu/titus/index.html,Court of TITUS's house. A banquet set out.,http://shakespeare.mit.edu/titus/titus.5.3.html,43.0,LUCIUS,"Some loving friends convey the emperor hence,\nAnd give him burial in his father's grave:\nMy father and Lavinia shall forthwith\nBe closed in our household's monument.\nAs for that heinous tiger, Tamora,\nNo funeral rite, nor man in mourning weeds,\nNo mournful bell shall ring her burial;\nBut throw her forth to beasts and birds of pre

In [120]:
df_all.to_csv('shakespeare.csv', sep='\t')

In [130]:
print(df_all.shape)

(108736, 10)


In [135]:
df_copy = df_all

### Get Characters Name Listed

In [131]:
len(df_all.character.unique())

966

### Remove Punctuation

In [136]:
df_all['line']=df_all.line.str.lower()
df_all['line']=df_all.line.str.replace("[^a-zA-Z]", " ",)

### Stemming

In [139]:
porter = PorterStemmer()
def tokenizer_porter(text):
    for word in text.split():
        return [porter.stem(word) for word in text.split()]
    
tokenizer_porter("This is another example this")

['Thi', 'is', 'anoth', 'exampl', 'thi']

In [156]:
all_stopwords = stopwords.words('english') #Add your own stopwords = + ['s','t']
df_all.line.values

# all_stopwords = set(stopwords.words('english')) #Add your own stopwords = + ['s','t']
# all_stopwords.update(['lt','p','\n',])

array(['in delivering my son from me  i bury a second husband ',
       'and i in going  madam  weep o er my father s death',
       'anew  but i must attend his majesty s command  to', ...,
       'by whom our heavy haps had their beginning ',
       'then  afterwards  to order well the state ',
       'that like events may ne er it ruinate '], dtype=object)

In [157]:
start_time = time.clock()
s = [x.lower() for x in all_stopwords]
for text in df_all.line.values:
    [x for x in text.split() if x.lower() not in s]
print (time.clock() - start_time)

start_time = time.clock()
s = set([x.lower() for x in all_stopwords])
for text in df_all.line.values:
    [x for x in text.split() if x.lower() not in s]
print (time.clock() - start_time)

AttributeError: 'NoneType' object has no attribute 'split'

### TF-IDF Vectorizing

In [158]:
vectorizer = TfidfVectorizer(tokenizer = tokenizer_porter,
                             stop_words=all_stopwords,
                             use_idf=False,
                             max_features = 100,ngram_range=(1,1))
train_data_features = vectorizer.fit_transform(df_all.line.values)
train_data_features = train_data_features.A #toarray()  
vocab = vectorizer.get_feature_names()

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(coef_PCA[:,0], coef_PCA[:,2], s = 70, zorder=0, c=dark2_colors[0])
for i in range(coef_PCA.shape[0]):
    plt.text(coef_PCA[i,0]+.1, coef_PCA[i,2], usarrests_scaled.ix[i]['State'], zorder=1, size=12, color=shade_black) 
plt.ylabel('3rd Principal component' )
plt.xlabel('1st Principal component' )
plt.title('PC3 vs. PC1')
remove_border()
plt.show()

fig = plt.figure(1, figsize=(10, 8))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=15, azim=150)
plt.cla()

ax.scatter(coef_PCA[:,0], coef_PCA[:,1], coef_PCA[:,2], s = 130, zorder=0, c=dark2_colors[0])
for i in range(coef_PCA.shape[0]):
    ax.text(coef_PCA[i,0], coef_PCA[i,1], coef_PCA[i,2], usarrests_scaled.ix[i]['State'], zorder=1, size=12, color=shade_black) 

ax.set_xlabel('1st Principal component')
ax.set_ylabel('2nd Principal component')
ax.set_zlabel('3rd Principal component')
ax.set_title('1st, 2nd and 3rd Principal Components')
plt.show()

def make_loading_plot(components, cols):
    summ_up = lambda x, y: abs(x) + abs(y)
    loadings = pd.DataFrame(data=dict(X=components[0], Y=components[1], name=cols))
    #loadings['summ'] = loadings.apply(lambda row: summ_up(row['X'], row['Y']), axis=1)
    plt.scatter(loadings.X, loadings.Y, s=200, label="Features", 
                facecolor=dark2_colors[1], edgecolor=shade_black, linewidth=0.35);

    for i in range(loadings.shape[0]):
        plt.text(loadings.ix[i]['X']+.01, loadings.ix[i]['Y'], loadings.ix[i]['name'], size=12, color=shade_black, fontsize =20)  

    plt.title("Feature loadings for the PCA");
    plt.xlabel("Feature loadings on PC1");
    plt.ylabel("Feature loadings on PC2");
    plt.grid(True)
    
    legend = plt.legend(loc=2,frameon=True,framealpha=0.6, scatterpoints=1)
    rect = legend.get_frame()
    rect.set_facecolor(light_grey)
    rect.set_linewidth(0.6)

make_loading_plot(pca.components_, usarrests_scaled.columns[1:])

# plt.show()


make_loading_plot(pca.components_, usarrests_scaled.columns[1:])
plt.scatter(coef_PCA[:,0], coef_PCA[:,1], s = 70, alpha=0.7, zorder=0, color = dark2_colors[0])
for i in range(coef_PCA.shape[0]):
    plt.text(coef_PCA[i,0]+.1, coef_PCA[i,1], usarrests_scaled.ix[i]['State'],  zorder=1, size=14, color=shade_black)  
plt.ylabel('2nd Principal component' )
plt.xlabel('1st Principal component' )
plt.show()